<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/INVESTIGAR_DADOS_covid_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Conexão com o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Instalação e importação de bibliotecas

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
!pip install darts 

In [ ]:
!pip install matplotlib==3.1.1

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import torch
import seaborn as sns 

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

from datetime import datetime
from darts import TimeSeries, timeseries

from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    NaiveMean,
    NaiveEnsembleModel,   
)

from darts.metrics import mape, mase, rmse
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
torch.manual_seed(1); np.random.seed(1)  # for reproducibility

##Seleção dos dados e leitura do dataset para casos

In [ ]:
#Seleciona o arquivo recentemente baixado da pasta de CSVs
pasta = '/content/drive/MyDrive/NPCA - COVID/_CASOS/DataSet/'
lista_de_arquivos = glob.glob(pasta + '*') 
#Seleciona o ultimo arquivo que foi criado e adicionado na pasta
arquivo_casos = max(lista_de_arquivos, key=os.path.getctime) 
arquivo_casos

In [ ]:
#Leitura do Arquivo
data_set_casos = pd.read_excel(arquivo_casos)
data_set_casos

##Seleção dos dados e leitura do dataset para óbitos

In [ ]:
#Seleciona o arquivo recentemente baixado da pasta de dataset
pasta1 = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/DataSet/'
lista_de_arquivos1 = glob.glob(pasta1 + '*') 
#Seleciona o ultimo arquivo que foi criado e adicionado na pasta
arquivo_obitos = max(lista_de_arquivos1, key=os.path.getctime) 
arquivo_obitos

'/content/drive/MyDrive/NPCA - COVID/_OBITOS/DataSet/Dataset_31_12_2022.xlsx'

In [ ]:
#Leitura do Arquivo
data_set_obitos = pd.read_excel(arquivo_obitos)
data_set_obitos

##Tratamento dos dados para as variáveis de casos e óbitos

In [ ]:
#Substitue valores negativos por valores iqual zero. 

data_set_casos['Casos_DataSintoma_MM_atual_PA'] = np.where(data_set_casos['Casos_DataSintoma_MM_atual_PA'] <= 0, 0.00001, data_set_casos['Casos_DataSintoma_MM_atual_PA'])
data_set_casos['Casos_Publicacao_MM_atual_PA'] = np.where(data_set_casos['Casos_Publicacao_MM_atual_PA'] <= 0, 0.00001, data_set_casos['Casos_Publicacao_MM_atual_PA'])
data_set_casos['Casos_DataSintoma_MM_7dias_PA'] = np.where(data_set_casos['Casos_DataSintoma_MM_7dias_PA'] <= 0, 0.00001, data_set_casos['Casos_DataSintoma_MM_7dias_PA'])
data_set_casos['Casos_Publicacao_MM_7dias_PA'] = np.where(data_set_casos['Casos_Publicacao_MM_7dias_PA'] <= 0, 0.00001, data_set_casos['Casos_Publicacao_MM_7dias_PA'])
data_set_casos['Casos_DataSintoma_MM_14dias_PA'] = np.where(data_set_casos['Casos_DataSintoma_MM_14dias_PA'] <= 0, 0.00001, data_set_casos['Casos_DataSintoma_MM_14dias_PA'])
data_set_casos['Casos_Publicacao_MM_14dias_PA'] = np.where(data_set_casos['Casos_Publicacao_MM_14dias_PA'] <= 0, 0.00001, data_set_casos['Casos_Publicacao_MM_14dias_PA'])
data_set_casos['Vacinacao_Dose1_%decimal_PA'] = np.where(data_set_casos['Vacinacao_Dose1_%decimal_PA'] <= 0, 0.00001, data_set_casos['Vacinacao_Dose1_%decimal_PA'])
data_set_casos['Vacinacao_Dose2_%decimal_PA'] = np.where(data_set_casos['Vacinacao_Dose2_%decimal_PA'] <= 0, 0.00001, data_set_casos['Vacinacao_Dose2_%decimal_PA'])
data_set_casos['Vacinacao_Dose3_%decimal_PA'] = np.where(data_set_casos['Vacinacao_Dose3_%decimal_PA'] <= 0, 0.00001, data_set_casos['Vacinacao_Dose3_%decimal_PA'])
data_set_casos

In [ ]:
#Substitue valores negativos e iguais a zer0 por valores decimais que não afetarão a análise dos dados. 

data_set_obitos['Obitos_DataOcorrencia_MM_atual_PA'] = np.where(data_set_obitos['Obitos_DataOcorrencia_MM_atual_PA'] <= 0, 0.00001, data_set_obitos['Obitos_DataOcorrencia_MM_atual_PA'])
data_set_obitos['Obitos_Publicacao_MM_atual_PA'] = np.where(data_set_obitos['Obitos_Publicacao_MM_atual_PA'] <= 0, 0.00001, data_set_obitos['Obitos_Publicacao_MM_atual_PA'])
data_set_obitos['Obitos_DataOcorrencia_MM_7dias_PA'] = np.where(data_set_obitos['Obitos_DataOcorrencia_MM_7dias_PA'] <= 0, 0.00001, data_set_obitos['Obitos_DataOcorrencia_MM_7dias_PA'])
data_set_obitos['Obitos_Publicacao_MM_7dias_PA'] = np.where(data_set_obitos['Obitos_Publicacao_MM_7dias_PA'] <= 0, 0.00001, data_set_obitos['Obitos_Publicacao_MM_7dias_PA'])
data_set_obitos['Obitos_DataOcorrencia_MM_14dias_PA'] = np.where(data_set_obitos['Obitos_DataOcorrencia_MM_14dias_PA'] <= 0, 0.00001, data_set_obitos['Obitos_DataOcorrencia_MM_14dias_PA'])
data_set_obitos['Obitos_Publicacao_MM_14dias_PA'] = np.where(data_set_obitos['Obitos_Publicacao_MM_14dias_PA'] <= 0, 0.00001, data_set_obitos['Obitos_Publicacao_MM_14dias_PA'])
data_set_obitos

##Instanciação das séries temporais principais e das variaveis secundarias para Casos

In [ ]:
#Converte coluna de data (strng) para datetime 
data_set_casos['data'] = pd.to_datetime(data_set_casos['data'])
#Converte o tipo de dado datetime para formato de data Ano-Mês-Dia
data_set_casos['data'] = data_set_casos['data'].values.astype('datetime64[D]')

# Seleciona as series temporais dropando as colunas
df_sintoma = data_set_casos.drop(data_set_casos.columns[2], axis=1, inplace=False)
serie_sintoma = TimeSeries.from_dataframe(df_sintoma,'data','Casos_DataSintoma_MM_atual_PA')
df_publicacao = data_set_casos.drop(data_set_casos.columns[1], axis=1, inplace=False)
serie_publicacao = TimeSeries.from_dataframe(df_publicacao,'data','Casos_Publicacao_MM_atual_PA')

#Gráfico com as informações das séries temporais
plt.figure(figsize = (12, 8))
serie_sintoma.plot(label='Casos de Covid por data de sintoma')
serie_publicacao.plot(label='Casos de Covid por data de publicacao')

In [ ]:
#Seleção das variáveis auxiliares
sintoma_7dias = data_set_casos.loc[:, ['data', 'Casos_DataSintoma_MM_7dias_PA']].fillna(0)
publicacao_7dias = data_set_casos.loc[:, ['data', 'Casos_Publicacao_MM_7dias_PA']].fillna(0)
sintoma_14dias = data_set_casos.loc[:, ['data', 'Casos_DataSintoma_MM_14dias_PA']].fillna(0)
publicacao_14dias = data_set_casos.loc[:, ['data', 'Casos_Publicacao_MM_14dias_PA']].fillna(0)

vacinacao_1Dose = data_set_casos.loc[:, ['data', 'Vacinacao_Dose1_%decimal_PA']].fillna(0)
vacinacao_2Dose = data_set_casos.loc[:, ['data', 'Vacinacao_Dose2_%decimal_PA']].fillna(0)
vacinacao_3Dose = data_set_casos.loc[:, ['data', 'Vacinacao_Dose3_%decimal_PA']].fillna(0)


In [ ]:
# criação das séries temporais auxiliares de sintoma e publicacao de 7 e 14 dias

serie_sintoma_7dias = TimeSeries.from_dataframe(sintoma_7dias,'data','Casos_DataSintoma_MM_7dias_PA')
serie_publicacao_7dias = TimeSeries.from_dataframe(publicacao_7dias,'data','Casos_Publicacao_MM_7dias_PA')
serie_sintoma_14dias = TimeSeries.from_dataframe(sintoma_14dias,'data','Casos_DataSintoma_MM_14dias_PA')
serie_publicacao_14dias = TimeSeries.from_dataframe(publicacao_14dias ,'data','Casos_Publicacao_MM_14dias_PA')

plt.figure(figsize = (10, 6))

serie_sintoma_7dias.plot()
serie_publicacao_7dias.plot()
serie_sintoma_14dias.plot()
serie_publicacao_14dias.plot()

In [ ]:
# criação das séries temporais auxiliares de vacinação
serie_vacinacao_1Dose = TimeSeries.from_dataframe(df=vacinacao_1Dose, time_col='data')
serie_vacinacao_2Dose = TimeSeries.from_dataframe(df=vacinacao_2Dose, time_col='data')
serie_vacinacao_3Dose = TimeSeries.from_dataframe(df=vacinacao_3Dose, time_col='data')

plt.figure(figsize = (12, 8))

serie_vacinacao_1Dose.plot()
serie_vacinacao_2Dose.plot()
serie_vacinacao_3Dose.plot()

##Instanciação das séries temporais principais para óbitos

In [ ]:
#Converte coluna de data (strng) para datetime 
data_set_obitos['data'] = pd.to_datetime(data_set_casos['data'])
#Converte o tipo de dado datetime para formato de data Ano-Mês-Dia
data_set_obitos['data'] = data_set_casos['data'].values.astype('datetime64[D]')

# Seleciona as series temporais dropando as colunas
df_obito_ocorrencia = data_set_obitos.drop(data_set_obitos.columns[2], axis=1, inplace=False).fillna(0)
serie_obito_ocorrencia = TimeSeries.from_dataframe(df_obito_ocorrencia,'data','Obitos_DataOcorrencia_MM_atual_PA')
df_obito_publicacao = data_set_obitos.drop(data_set_obitos.columns[1], axis=1, inplace=False).fillna(0)
serie_obito_publicacao = TimeSeries.from_dataframe(df_obito_publicacao,'data','Obitos_Publicacao_MM_atual_PA')

#Gráfico com as informações das séries temporais
plt.figure(figsize = (12, 8))
serie_obito_ocorrencia.plot(label='Obitos por data de Ocorrência')
serie_obito_publicacao.plot(label='Obitos por data de Publicacao')

## Estatística descritiva para Casos e Obitos
(media, desviopadrão, minimo, máximo)


In [ ]:
#Calcula a estatística descritiva de todas as variaveis do dataset
statistic = data_set_obitos.loc[:, ['Obitos_DataOcorrencia_MM_atual_PA', 'Obitos_Publicacao_MM_atual_PA','Casos_DataSintoma_MM_atual_PA', 'Casos_Publicacao_MM_atual_PA']]
statistic.rename(columns = {'Obitos_DataOcorrencia_MM_atual_PA': 'Deaths_Occurrence_date', 'Obitos_Publicacao_MM_atual_PA': 'Deaths_Publication_date', 'Casos_DataSintoma_MM_atual_PA': 'Cases_Symptom_date', 'Casos_Publicacao_MM_atual_PA':'Cases_Publication_date'}, inplace=True)
x = statistic.describe()
x.to_excel("describe.xlsx", index = True, encoding = 'utf-8')
x

Matriz de correlação de Pearson

In [ ]:
#calcula a correlacao de pearson entre as variaveis principais e de vacinação
corr_df = data_set_obitos.loc[:, ['Casos_DataSintoma_MM_atual_PA','Casos_Publicacao_MM_atual_PA','Obitos_DataOcorrencia_MM_atual_PA', 
                                   'Obitos_Publicacao_MM_atual_PA','Vacinacao_Dose1_%decimal_PA', 'Vacinacao_Dose2_%decimal_PA', 'Vacinacao_Dose3_%decimal_PA']]
corr = corr_df.corr(method='pearson')

#cria uma matriz de correlaçao entre as colunas da tabela do dataset
plt.figure(figsize=(10, 8))
ax = sns.heatmap(corr, cmap = "coolwarm", center = 0,  annot=True)
ax.get_ylim()
ax.set_ylim(7.0, 0)
figs = ax.get_figure()
figs.savefig('/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/matrizdecorrelacao.png', dpi=300, transparent=False, bbox_inches='tight')
plt.show()

##Visualização Inicial

In [ ]:
# Seleciona os ultimos 7 dias para inspecionar sazonalidade e fazer prediçoes com os modelos ingenuos
num_dias_val = 7
train_sintoma, val_sintoma = serie_sintoma[:-num_dias_val], serie_sintoma[-num_dias_val:]
train_publicacao, val_publicacao = serie_publicacao[:-num_dias_val], serie_publicacao[-num_dias_val:]
train_obito_ocorrencia, val_obito_ocorrencia = serie_obito_ocorrencia[:-num_dias_val], serie_obito_ocorrencia[-num_dias_val:]
train_obito_publicacao, val_obito_publicacao =  serie_obito_publicacao[:-num_dias_val], serie_obito_publicacao[-num_dias_val:]

#plota a serie com os casos por data do sintoma e publicacao dos ultimos 7 dias
fig, ax = plt.subplots(figsize=(12,8));
fig.suptitle('Média móvel de casos de COVID-19 no estado do Pará - últimos '+str(num_dias_val)+' dias', fontsize=12)
val_sintoma.plot(label='Casos por Data de Publicação')
val_publicacao.plot(label='Casos por Data de Publicação')
val_obito_ocorrencia.plot(label='Obitos por Data de Ocorrencia')
val_obito_publicacao.plot(label='Obitos por Data de publicacao')
ax.xaxis.set_major_formatter(DateFormatter("%d/%m/%y"))

##Análise das series temporais

In [ ]:
#Listas para armazenar os valores de sazonalidade
lista_casos_sintoma = []
lista_casos_publicacao = []
lista_obitos_ocorrencia = []
lista_obitos_publicacao = []

In [ ]:
#Verificar se existe sazonalidade nas series
def verificar_sazonalidade(serie_temporal, list):
  list.clear()
  for m in range(2, 365):
      sazonal, periodo = check_seasonality(serie_temporal, m = m, max_lag = 365, alpha=0.05)
      if sazonal:
       print('Há sazonalidade de ordem {}.'.format(periodo))
       list.append(periodo)
       plot_acf(serie_temporal, m = m, max_lag = 365, alpha = 0.05)

In [ ]:
verificar_sazonalidade(serie_sintoma, lista_casos_sintoma)
lista_casos_sintoma

In [ ]:
verificar_sazonalidade(serie_publicacao, lista_casos_publicacao)
lista_casos_publicacao

In [ ]:
verificar_sazonalidade(serie_obito_ocorrencia, lista_obitos_ocorrencia)
lista_obitos_ocorrencia

In [ ]:
verificar_sazonalidade(serie_obito_publicacao, lista_obitos_publicacao)
lista_obitos_publicacao

##Aplicar modelos ingénuos

**Treinos com o Modelo Ingênuo NaiveSeasonal**

In [ ]:
predict_caso_sintoma = []
for x in lista_casos_sintoma:
      NaiveSeasonal_model = NaiveSeasonal(K = x)
      NaiveSeasonal_model.fit(train_sintoma)
      y = NaiveSeasonal_model.predict(num_dias_val)
      description = ('Casos_Data_Sintoma')
      z = (x, y, description)
      predict_caso_sintoma.append(z)

In [ ]:
predict_caso_publicacao = []
for x in lista_casos_publicacao:
      NaiveSeasonal_model = NaiveSeasonal(K = x)
      NaiveSeasonal_model.fit(train_publicacao)
      y = NaiveSeasonal_model.predict(num_dias_val)
      description = ('Casos_Data_Publicacao')
      z = (x, y, description)
      predict_caso_publicacao.append(z)

In [ ]:
predict_obito_ocorrencia = []
for x in lista_obitos_ocorrencia:
      NaiveSeasonal_model = NaiveSeasonal(K = x)
      NaiveSeasonal_model.fit(train_obito_ocorrencia)
      y = NaiveSeasonal_model.predict(num_dias_val)
      description = ('Obitos_Data_Ocorrencia')
      z = (x, y, description)
      predict_obito_ocorrencia.append(z)

In [ ]:
predict_obito_publicacao = []
for x in lista_obitos_publicacao:
      NaiveSeasonal_model = NaiveSeasonal(K = x)
      NaiveSeasonal_model.fit(train_obito_publicacao)
      y = NaiveSeasonal_model.predict(num_dias_val)
      description = ('Obitos_Data_Publicacao')
      z = (x, y, description)
      predict_obito_publicacao.append(z)

In [ ]:
ns_casos_sintoma = predict_caso_sintoma

In [ ]:
ns_casos_publicacao = predict_caso_publicacao

In [ ]:
ns_obitos_ocorrencia = predict_obito_ocorrencia

In [ ]:
ns_obitos_publicacao = predict_obito_publicacao

**Treino com o modelo NaiveDrift**

In [ ]:
#Função para os treinos com o modelo NaiveDrift
NaiveDrift_model = NaiveDrift()
def treinar_NaiveDrift (serie_temporal, int):
    NaiveDrift_model.fit(serie_temporal)
    y = NaiveDrift_model.predict(int)
    return y

In [ ]:
#Treinos para casos por data de sintoma, de publicacao e para obitos por data de ocorrencia e publicacao utilizando a função de treino com o NaiveDrift
nd_casos_sintoma = treinar_NaiveDrift(train_sintoma, num_dias_val)
nd_casos_publicacao = treinar_NaiveDrift(train_publicacao, num_dias_val)
nd_obitos_ocorrencia = treinar_NaiveDrift(train_obito_ocorrencia, num_dias_val)
nd_obitos_publicacao = treinar_NaiveDrift(train_obito_publicacao, num_dias_val)

**Treino com o modelo NaiveMean**

In [ ]:
#Função para os treinos com o modelo NaiveMean
NaiveMean_model = NaiveMean()
def treinar_NaiveMean (serie_temporal, int):
    NaiveMean_model.fit(serie_temporal)
    y = NaiveMean_model.predict(int)
    return y

In [ ]:
#Treinos para casos por data de sintoma, de publicacao e para obitos por data de ocorrencia e publicacao utilizando a função de treino com o NaiveMean
nm_casos_sintoma = treinar_NaiveMean(train_sintoma, num_dias_val)
nm_casos_publicacao = treinar_NaiveMean(train_publicacao, num_dias_val)
nm_obitos_ocorrencia = treinar_NaiveMean(train_obito_ocorrencia, num_dias_val)
nm_obitos_publicacao = treinar_NaiveMean(train_obito_publicacao, num_dias_val)

**Treino com modelo NaiveEnsemble**

In [ ]:
#Esse ensemble faz somente a media da projeção dos modelos
NaiveEnsemble_model = NaiveEnsembleModel(models=[NaiveSeasonal(), NaiveDrift(), NaiveMean()])
def modelo_Ensemble (serie_temporal, int):
    NaiveEnsemble_model.fit(serie_temporal)
    y = NaiveEnsemble_model.predict(int)
    return y

In [ ]:
#Treino da media de projeção dos modelos utilizando a função do modelo Ensemble
ne_casos_sintoma = modelo_Ensemble(train_sintoma, num_dias_val)
ne_casos_publicacao = modelo_Ensemble(train_publicacao, num_dias_val)
ne_obitos_ocorrencia = modelo_Ensemble(train_obito_ocorrencia, num_dias_val)
ne_obitos_publicacao = modelo_Ensemble(train_obito_publicacao, num_dias_val)

**Vizualização das predições**

In [ ]:
#Função para plotar as predições do modelo NaiveSeasonal
def plotar_NaiveSeasonal (list):
      for x in list:
        a, b, c = x
        b.plot(label='NaiveSeasonal Forecast for '+str(c)+' with K='+str(a))

In [ ]:
fig, ax = plt.subplots(figsize=(12,10));
fig.suptitle('Prediction of COVID-19 cases by Symptom Date - '+str(num_dias_val)+' days', fontsize=10);
val_sintoma.plot(label='Actual')
plotar_NaiveSeasonal(ns_casos_sintoma)
nd_casos_sintoma.plot(label='NaiveDrift Forecast')
nm_casos_sintoma.plot(label='NaiveMean Forecast')
ne_casos_sintoma.plot(label='NaiveEnsemble Forecast')
ax.xaxis.set_major_formatter(DateFormatter("%d/%m/%y"))
fig.savefig('/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/Naive_casos_data_sintoma.png', dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(13,10));
fig.suptitle('Prediction of COVID-19 Cases by Publication Date - '+str(num_dias_val)+' days', fontsize=10);
val_publicacao.plot(label='Actual')
plotar_NaiveSeasonal(ns_casos_publicacao)
nd_casos_publicacao.plot(label='NaiveDrift Forecast')
nm_casos_publicacao.plot(label='NaiveMean Forecast')
ne_casos_publicacao.plot(label='NaiveEnsemble Forecast')
ax.xaxis.set_major_formatter(DateFormatter("%d/%m/%y"))
fig.savefig('/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/Naive_casos_data_publicacao.png', dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(13,10));
fig.suptitle('Prediction of deaths from COVID-19 by Date of Occurrence - '+str(num_dias_val)+' days', fontsize=10);
val_obito_ocorrencia.plot(label='Actual')
plotar_NaiveSeasonal(ns_obitos_ocorrencia)
nd_obitos_ocorrencia.plot(label='NaiveDrift Forecast')
nm_obitos_ocorrencia.plot(label='NaiveMean Forecast')
ne_obitos_ocorrencia.plot(label='NaiveEnsemble Forecast')
ax.xaxis.set_major_formatter(DateFormatter("%d/%m/%y"))
fig.savefig('/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/Naive_obitos_data_ocorrencia.png', dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(13,10));
fig.suptitle('Prediction of deaths from COVID-19 by Publication Date - '+str(num_dias_val)+' days', fontsize=10);
val_obito_publicacao.plot(label='Actual')
plotar_NaiveSeasonal(ns_obitos_publicacao)
nd_obitos_publicacao.plot(label='NaiveDrift Forecast')
nm_obitos_publicacao.plot(label='NaiveMean Forecast')
ne_obitos_publicacao.plot(label='NaiveEnsemble Forecast')
ax.xaxis.set_major_formatter(DateFormatter("%d/%m/%y"))
fig.savefig('/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/Naive_obitos_data_publicacao.png', dpi=300, transparent=False, bbox_inches='tight')

##Métricas de erro

In [ ]:
#Funções para calcular as métricas do modelo NaiveSeasonal
def mape_NaiveSeasonal (serie_temporal, list):
    for x in list:
     a, b, c = x
     NaiveSeasonal_model = NaiveSeasonal(K=a)
     print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveSeasonal_model, mape(serie_temporal, b)))

def rmse_NaiveSeasonal (serie_temporal, list):
    for x in list:
     a, b, c = x
     NaiveSeasonal_model = NaiveSeasonal(K=a)
     print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveSeasonal_model, rmse(serie_temporal, b)))

Porcentagens das Métricas de erro para casos por data de sintoma

In [ ]:
mape_NaiveSeasonal(val_sintoma, ns_casos_sintoma)
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveDrift_model, mape(val_sintoma, nd_casos_sintoma)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveMean_model, mape(val_sintoma, nm_casos_sintoma)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveEnsemble_model, mape(val_sintoma, ne_casos_sintoma)))

In [ ]:
rmse_NaiveSeasonal(val_sintoma, ns_casos_sintoma)
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveDrift_model, rmse(val_sintoma, nd_casos_sintoma)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveMean_model, rmse(val_sintoma, nm_casos_sintoma)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveEnsemble_model, rmse(val_sintoma, ne_casos_sintoma)))

Porcentagens das Métricas de erro para casos por data de publicação

In [ ]:
mape_NaiveSeasonal(val_publicacao, ns_casos_publicacao)
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveDrift_model, mape(val_publicacao, nd_casos_publicacao)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveMean_model, mape(val_sintoma, nm_casos_publicacao)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveEnsemble_model, mape(val_sintoma, ne_casos_publicacao)))

In [ ]:
rmse_NaiveSeasonal(val_publicacao, ns_casos_publicacao)
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveDrift_model, rmse(val_publicacao, nd_casos_publicacao)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveMean_model, rmse(val_publicacao, nm_casos_publicacao)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveEnsemble_model, rmse(val_publicacao, ne_casos_publicacao)))

Porcentagens das Métricas de erro para óbitos por data de ocorrencia

In [ ]:
mape_NaiveSeasonal(val_obito_ocorrencia, ns_obitos_ocorrencia)
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveDrift_model, mape(val_obito_ocorrencia, nd_obitos_ocorrencia)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveMean_model, mape(val_obito_ocorrencia, nm_obitos_ocorrencia)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveEnsemble_model, mape(val_obito_ocorrencia, ne_obitos_ocorrencia)))

In [ ]:
rmse_NaiveSeasonal(val_obito_ocorrencia, ns_obitos_ocorrencia)
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveDrift_model, rmse(val_obito_ocorrencia, nd_obitos_ocorrencia)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveMean_model, rmse(val_obito_ocorrencia, nm_obitos_ocorrencia)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveEnsemble_model, rmse(val_obito_ocorrencia, ne_obitos_ocorrencia)))

Porcentagens das Métricas de erro para óbitos por data de publicação

In [ ]:
mape_NaiveSeasonal(val_obito_publicacao, ns_obitos_publicacao)
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveDrift_model, mape(val_obito_publicacao, nd_obitos_publicacao)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveMean_model, mape(val_obito_publicacao, nm_obitos_publicacao)))
print('Modelo {} obteve MAPE: {:.2f}%'.format(NaiveEnsemble_model, mape(val_obito_publicacao, ne_obitos_publicacao)))

In [ ]:
rmse_NaiveSeasonal(val_obito_publicacao, ns_obitos_publicacao)
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveDrift_model, rmse(val_obito_publicacao, nd_obitos_publicacao)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveMean_model, rmse(val_obito_publicacao, nm_obitos_publicacao)))
print('Modelo {} obteve RMSE: {:.2f}'.format(NaiveEnsemble_model, rmse(val_obito_publicacao, ne_obitos_publicacao)))

Continua no script de TREINAR MODELOS.

Link:  https://colab.research.google.com/drive/1oIdU_g1U0IDrsOvJxumXP5-wES2Fcius